# BEBOP and QM Descriptors for Deblocking Temperatures

**Description:** This notebook calculates electronic descriptors from BEBOP methods and QM models. It extracts bond energies, hybridization energies, and resonance energies from BEBOP calculations and nucleophilicity indices and deprotonation energies from QM methods for a series of capping agents and capped adducts of MDI.

**Reference:** Remsha Rafiq, Ramakrishna Suresh, Barbaro Zulueta, Jessica Gondak, Hannah Zucco, Jason E.
Shoemaker, John A. Keith, Goetz Veser. Bond Energy Descriptors Enable Machine Learning with Limited Data: Design of Thermoplastic Polyurethane Recycling Agents **2025** (*to be submitted*)

**Repository:** https://github.com/BLZ11/deblocking_temp

---

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Resonance Bond Definitions](#resonance)
3. [BEBOP Bond Energy Calculations](#bebop)
4. [Nucleophilicity and HOMO-LUMO Gap Calculations](#nucleophilicity)
5. [Deprotonation Energy Calculations](#deprotonation)
6. [Export Results](#export)

<a id='setup'></a>
## 1. Setup and Configuration

Import required packages and configure paths. Users should modify the `DATA_PATH` variable to point to their Gaussian output files.

In [ ]:
"""Setup and imports for deblocking descriptor calculations."""

from __future__ import annotations

import os
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field

import numpy as np
import matplotlib.pyplot as plt

# BEBOP import - adjust path as needed for your installation
# pip install bebop or add to sys.path
try:
    from bebop_g1 import BEBOP
except ImportError:
    raise ImportError(
        "BEBOP package not found. Please install via pip or add to sys.path.\n"
        "See: https://github.com/keithgroup/bebop-qc for installation instructions."
    )

# ============================================================================
# USER CONFIGURATION - Modify these paths for your system
# ============================================================================

DATA_PATH = Path("/path/to/your/data")  # Root folder containing Gaussian outputs
GAUSSIAN_B3LYP_CBSB7 = DATA_PATH / "gaussian_outputs" / "b3lyp_cbsb7"
GAUSSIAN_B3LYP_631G = DATA_PATH / "gaussian_outputs" / "b3lyp_631g_star"
GAUSSIAN_G4MP2 = DATA_PATH / "gaussian_outputs" / "g4mp2"

# Output directory for results
OUTPUT_DIR = Path("./results")
OUTPUT_DIR.mkdir(exist_ok=True)

# Physical constants
HARTREE_TO_KCAL = 627.5096  # kcal/mol per Hartree
HARTREE_TO_EV = 27.211      # eV per Hartree

<a id='resonance'></a>
## 2. Resonance Bond Definitions

Define the atom pairs participating in resonance for each molecular system. These are used to calculate resonance stabilization energies via BEBOP.

**Bond type notation:**
- `C-N*`: C-N bonds participating in resonance (when the lone pairs of N are part of it)
- `C=N`: Carbon-nitrogen double bonds
- `N-N`: Nitrogen-nitrogen single bonds
- etc.

**Atom indices:** 0-indexed from Gaussian output files.

In [ ]:
"""Resonance bond definitions for each molecular system.

Format: {molecule_name: {bond_type: [(atom_i, atom_j), ...]}}

Bond types:
    C-N*  : C-N bonds in resonance (delocalized)
    C=N   : C-N double bonds
    C-C   : C-C single bonds
    C=C   : C-C double bonds
    N-N   : N-N single bonds
    N=N   : N-N double bonds
    N-O   : N-O single bonds
    C-O   : C-O single bonds
"""

RESONANCE_BONDS: Dict[str, Dict[str, List[Tuple[int, int]]]] = {
    # Capped Adduct Molecules
    "124triazole": {
        "C-N*": [(8, 19), (19, 20), (20, 21), (22, 23)],
        "C=N": [(21, 22)],
        "N-N": [(19, 23)],
    },
    "3methylpyrazole": {
        "C=N": [(8, 19), (22, 23)],
        "C=C": [(20, 21)],
        "C-C": [(21, 22)],
        "C-N*": [(19, 20)],
        "N-N": [(19, 23)],
    },
    "acetoneoxime": {
        "C=N": [(20, 21)],
        "N-O": [(19, 20)],
        "C-O": [(8, 19)],
    },
    "acetophenoneoxime": {
        "C-C": [(26, 27), (28, 29), (30, 31), (21, 26)],
        "C=C": [(26, 31), (27, 28), (29, 30)],
        "C=N": [(20, 21)],
        "N-O": [(19, 20)],
        "C-O": [(8, 19)],
    },
    "benzotriazole": {
        "C=C": [(20, 21), (25, 26), (23, 24)],
        "C-C": [(20, 23), (21, 26), (24, 25)],
        "C-N*": [(19, 20), (21, 27), (8, 19)],
        "N-N": [(19, 22)],
        "N=N": [(22, 27)],
    },
    "benzylalcohol": {
        "C-C": [(20, 21), (22, 23), (24, 25)],
        "C=C": [(21, 22), (23, 24), (20, 25)],
    },
    "naphthylEthanol": {
        "C=C": [(25, 31), (29, 30), (27, 28), (34, 37), (32, 38)],
        "C-C": [(30, 31), (28, 29), (25, 27), (27, 34), (37, 38), (28, 32)],
    },
    "nmethylaniline": {
        "C-C": [(20, 25), (26, 27), (28, 29)],
        "C=C": [(25, 26), (27, 28), (20, 29)],
        "C-N*": [(19, 20), (8, 19)],
    },
    "phenol": {
        "C=C": [(20, 21), (22, 23), (24, 25)],
        "C-C": [(21, 22), (23, 24), (20, 25)],
        "C-O": [(19, 20), (8, 19)],
    },
    "phenylEthanol": {
        "C-C": [(20, 21), (22, 23), (24, 25)],
        "C=C": [(21, 22), (23, 24), (20, 25)],
    },
    "pyrazole": {
        "C-C": [(21, 22)],
        "C=C": [(20, 21)],
        "C-N*": [(19, 20), (8, 19)],
        "C=N": [(22, 23)],
        "N-N": [(19, 23)],
    },
    "butanoneoxime": {"N-O": [(19, 20)], "C=N": [(20, 21)]},
    "cyclohexanoneoxime": {"N-O": [(19, 20)], "C=N": [(20, 21)]},
    "octanoneoxime": {"N-O": [(19, 20)], "C=N": [(20, 21)]},
    
    # Caping agents molecules
    "124triazole_cap": {
        "C-N*": [(2, 3), (3, 4), (5, 6)],
        "C=N": [(4, 5)],
        "N-N": [(2, 6)],
    },
    "3methylpyrazole_cap": {
        "C=N": [(5, 6)],
        "C=C": [(3, 4)],
        "C-C": [(4, 5)],
        "C-N*": [(2, 3)],
        "N-N": [(2, 6)],
    },
    "acetoneoxime_cap": {"C=N": [(3, 4)], "N-O": [(2, 3)]},
    "acetophenoneoxime_cap": {
        "C-C": [(4, 9), (9, 10), (13, 14), (11, 12)],
        "C=C": [(10, 11), (9, 14), (12, 13)],
        "C=N": [(3, 4)],
        "N-O": [(2, 3)],
    },
    "benzotriazole_cap": {
        "C=C": [(3, 4), (6, 7), (8, 9)],
        "C-C": [(3, 6), (7, 8), (4, 9)],
        "C-N*": [(2, 3), (4, 10)],
        "N-N": [(2, 5)],
        "N=N": [(5, 10)],
    },
    "benzylalcohol_cap": {
        "C-C": [(4, 5), (6, 7), (3, 8)],
        "C=C": [(3, 4), (5, 6), (7, 8)],
    },
    "naphthylEthanol_cap": {
        "C=C": [(8, 10), (11, 12), (13, 14), (15, 21), (17, 20)],
        "C-C": [(20, 21), (11, 15), (10, 17), (10, 11), (12, 13), (8, 14)],
    },
    "nmethylaniline_cap": {
        "C-C": [(11, 12), (3, 8), (9, 10)],
        "C=C": [(10, 11), (3, 12), (8, 9)],
        "C-N*": [(2, 3)],
    },
    "phenol_cap": {
        "C=C": [(5, 6), (7, 8), (3, 4)],
        "C-C": [(4, 5), (6, 7), (3, 8)],
        "C-O": [(2, 3)],
    },
    "phenylEthanol_cap": {
        "C-C": [(7, 8), (5, 6), (3, 4)],
        "C=C": [(3, 8), (6, 7), (4, 5)],
    },
    "pyrazole_cap": {
        "C-C": [(4, 5)],
        "C=C": [(3, 4)],
        "C-N*": [(2, 3)],
        "C=N": [(5, 6)],
        "N-N": [(2, 6)],
    },
    "butanoneoxime_cap": {"N-O": [(2, 3)], "C=N": [(3, 4)]},
    "cyclohexanoneoxime_cap": {"N-O": [(2, 3)], "C=N": [(3, 4)]},
    "octanoneoxime_cap": {"N-O": [(2, 3)], "C=N": [(3, 4)]},
}

In [ ]:
"""List of all molecular structures to analyze."""

# All structures: full molecules and their capped (blocking group only) versions
STRUCTURES: List[str] = [
    # Triazole and pyrazole derivatives
    "124triazole", "124triazole_cap",
    "3methylpyrazole", "3methylpyrazole_cap",
    "pyrazole", "pyrazole_cap",
    "benzotriazole", "benzotriazole_cap",
    # Oximes
    "acetoneoxime", "acetoneoxime_cap",
    "acetophenoneoxime", "acetophenoneoxime_cap",
    "butanoneoxime", "butanoneoxime_cap",
    "cyclohexanoneoxime", "cyclohexanoneoxime_cap",
    "octanoneoxime", "octanoneoxime_cap",
    # Alcohols
    "benzylalcohol", "benzylalcohol_cap",
    "phenylEthanol", "phenylEthanol_cap",
    "naphthylEthanol", "naphthylEthanol_cap",
    "phenol", "phenol_cap",
    "ethanol", "ethanol_cap",
    "butanol", "butanol_cap",
    "hexanol", "hexanol_cap",
    "decanol", "decanol_cap",
    "cyclohexanol_axial", "cyclohexanol_axial_cap",
    "cyclohexanol_equatorial", "cyclohexanol_equatorial_cap",
    # Amines
    "nmethylaniline", "nmethylaniline_cap",
    "butylamine", "butylamine_cap",
    "hexylamine", "hexylamine_cap",
    "decylamine", "decylamine_cap",
    # Others
    "hydroxyethylmethacrylate", "hydroxyethylmethacrylate_cap",
    "hydroxymethylpentanone", "hydroxymethylpentanone_cap",
]

# Structures with defined resonance (subset of STRUCTURES)
RESONANCE_STRUCTURES = set(RESONANCE_BONDS.keys())

<a id='bebop'></a>
## 3. BEBOP Bond Energy Calculations

Extract bond energies and hybridization indices using the BEBOP method. For each structure, we calculate:

- **Gross bond energy**: Total bonding interaction (i.e., EHT and short-range repulsion)
- **Net bond energy**: Gross bond + environmental factors (i.e., hybridization)
- **Hybridization energy**: sp-character of atoms involved in key bonds
- **Resonance energy**: Stabilization from delocalized bonding

In [ ]:
@dataclass
class BEBOPResult:
    """Container for BEBOP calculation results for a single molecule.
    
    Attributes:
        name: Molecule identifier
        total_energy: Total molecular energy (kcal/mol)
        bond_co: Bond type string for C-O or C-N bond (e.g., "C-O")
        bond_nh: Bond type string for N-H bond ("None" for capped structures)
        gross_be_co: Gross bond energy for C-O/C-N bond (kcal/mol)
        net_be_co: Net bond energy for C-O/C-N bond (kcal/mol)
        gross_be_nh: Gross bond energy for N-H bond (kcal/mol)
        net_be_nh: Net bond energy for N-H bond (kcal/mol)
        hybridization: [C_hyb, N/O_hyb, N_hyb] indices for key atoms
        resonance_energy: Resonance stabilization energy (kcal/mol)
    """
    name: str
    total_energy: float
    bond_co: str
    bond_nh: str
    gross_be_co: float
    net_be_co: float
    gross_be_nh: float
    net_be_nh: float
    hybridization: List[float] = field(default_factory=lambda: [0.0, 0.0, 0.0])
    resonance_energy: float = 0.0

In [ ]:
def run_bebop_calculation(structure: str, gaussian_path: Path) -> BEBOPResult:
    """Run BEBOP analysis on a Gaussian output file.
    
    Args:
        structure: Name of the molecular structure
        gaussian_path: Path to directory containing Gaussian .out files
        
    Returns:
        BEBOPResult containing all calculated descriptors
        
    Notes:
        For full molecules: extracts bonds at indices (7,18) for C-O/C-N and (6,9) for N-H
        For capped molecules: uses indices (0,1) and sets N-H values to zero
    """
    output_file = gaussian_path / f"{structure}.out"
    data = BEBOP(str(output_file))
    
    # Get bond energy matrices
    be = data.bond_E(GrossBond=True, Composite=True)
    is_capped = structure.endswith("_cap")
    
    if is_capped:
        # Capping agent 
        bond_co = f"{data.mol[1]}-{data.mol[0]}"
        bond_nh = "None"
        gross_be_co = 0.0
        net_be_co = 0.0
        gross_be_nh = 0.0
        net_be_nh = 0.0
        # Hybridization: [C, N/O, N] - only N/O available for capped
        hybridization = [0.0, be["CompositeTable"][1][1], 0.0]
    else:
        # Capped adduct
        bond_co = f"{data.mol[7]}-{data.mol[18]}"
        bond_nh = f"{data.mol[6]}-{data.mol[9]}"
        gross_be_co = be["GrossBond"][18][7]
        net_be_co = be["NetBond"][18][7]
        gross_be_nh = be["GrossBond"][9][6]
        net_be_nh = be["NetBond"][9][6]
        # Hybridization: [C from C-O/C-N, N/O from C-O/C-N, N from N-H]
        hybridization = [
            be["CompositeTable"][7][7],
            be["CompositeTable"][18][18],
            be["CompositeTable"][6][6],
        ]
    
    # Calculate resonance energy if available
    resonance_energy = 0.0
    if structure in RESONANCE_STRUCTURES:
        resonance_energy = np.abs(data.resonance_E(RESONANCE_BONDS[structure]))
    
    return BEBOPResult(
        name=structure,
        total_energy=data.total_E(),
        bond_co=bond_co,
        bond_nh=bond_nh,
        gross_be_co=gross_be_co,
        net_be_co=net_be_co,
        gross_be_nh=gross_be_nh,
        net_be_nh=net_be_nh,
        hybridization=hybridization,
        resonance_energy=resonance_energy,
    )

In [ ]:
def run_all_bebop_calculations(
    structures: List[str], 
    gaussian_path: Path,
    verbose: bool = True
) -> List[BEBOPResult]:
    """Run BEBOP calculations for all structures.
    
    Args:
        structures: List of structure names to analyze
        gaussian_path: Path to Gaussian output directory
        verbose: If True, print progress
        
    Returns:
        List of BEBOPResult objects
    """
    results = []
    n_structures = len(structures)
    
    for i, structure in enumerate(structures, 1):
        if verbose:
            print(f"Processing {structure} ({i}/{n_structures})...", end="\r")
        
        try:
            result = run_bebop_calculation(structure, gaussian_path)
            results.append(result)
        except FileNotFoundError:
            print(f"\nWarning: File not found for {structure}")
        except Exception as e:
            print(f"\nError processing {structure}: {e}")
    
    if verbose:
        print(f"\nCompleted {len(results)}/{n_structures} calculations.")
    
    return results

In [ ]:
# Run BEBOP calculations for all structures
bebop_results = run_all_bebop_calculations(STRUCTURES, GAUSSIAN_B3LYP_CBSB7)

<a id='nucleophilicity'></a>
## 4. Nucleophilicity and HOMO-LUMO Gap Calculations

Calculate nucleophilicity index ($N$) relative to tetracyanoethylene (TCE) as reference:

$$N = E_{\text{HOMO}}^{\text{Nuc}} - E_{\text{HOMO}}^{\text{TCE}}$$

This is the Domingo's group nucleophilicity scale referenced to TCE.

The HOMO-LUMO gap is calculated as: 

$$E_{\text{HOMO-LUMO}} = E_{\text{HOMO}} - E_{\text{LUMO}}$$

In [ ]:
@dataclass
class OrbitalEnergies:
    """Container for molecular orbital energies.
    
    Attributes:
        homo: HOMO energy (eV)
        lumo: LUMO energy (eV)
        homo_lumo_gap: HOMO-LUMO gap (eV)
    """
    homo: float
    lumo: float
    
    @property
    def homo_lumo_gap(self) -> float:
        """HOMO-LUMO gap in eV."""
        return self.homo - self.lumo

In [ ]:
def parse_homo_lumo(filepath: Path) -> OrbitalEnergies:
    """Extract HOMO and LUMO energies from a Gaussian output file.
    
    Searches for the final set of orbital energies after geometry optimization
    converges (after 'Stationary point found').
    
    Args:
        filepath: Path to Gaussian .out file
        
    Returns:
        OrbitalEnergies with HOMO and LUMO in eV
        
    Raises:
        ValueError: If orbital energies cannot be parsed
    """
    occupied_energies: List[float] = []
    virtual_energies: List[float] = []
    
    with open(filepath, "r") as f:
        lines = f.readlines()
    
    found_stationary = False
    found_electronic_state = False
    
    for i, line in enumerate(lines):
        # Find stationary point (optimization complete)
        if "-- Stationary point found." in line and not found_stationary:
            found_stationary = True
            continue
        
        if not found_stationary:
            continue
            
        # Find electronic state section
        if "The electronic state is" in line:
            found_electronic_state = True
            continue
        
        if not found_electronic_state:
            continue
            
        # Stop at Mulliken population analysis
        if "Condensed to atoms (all electrons):" in line:
            break
        
        # Parse orbital energies
        if line.strip().startswith("Alpha  occ."):
            values = line[28:].split()
            occupied_energies.extend(float(v) for v in values)
        elif line.strip().startswith("Alpha virt."):
            values = line[28:].split()
            virtual_energies.extend(float(v) for v in values)
    
    if not occupied_energies or not virtual_energies:
        raise ValueError(f"Could not parse orbital energies from {filepath}")
    
    # Convert from Hartrees to eV
    homo = occupied_energies[-1] * HARTREE_TO_EV
    lumo = virtual_energies[0] * HARTREE_TO_EV
    
    return OrbitalEnergies(homo=homo, lumo=lumo)

In [ ]:
def calculate_nucleophilicity(
    e_homo_nucleophile: float, 
    e_homo_tce: float
) -> float:
    """Calculate nucleophilicity index relative to TCE.
    
    Args:
        e_homo_nucleophile: HOMO energy of nucleophile (eV)
        e_homo_tce: HOMO energy of TCE reference (eV)
        
    Returns:
        Nucleophilicity index N (eV)
    """
    return e_homo_nucleophile - e_homo_tce

In [ ]:
@dataclass
class NucleophilicityResult:
    """Nucleophilicity calculation results for a molecule.
    
    Attributes:
        name: Molecule identifier
        homo: HOMO energy (eV)
        lumo: LUMO energy (eV)
        homo_lumo_gap: HOMO-LUMO gap (eV)
        nucleophilicity: N index relative to TCE (eV)
    """
    name: str
    homo: float
    lumo: float
    homo_lumo_gap: float
    nucleophilicity: float

In [ ]:
def calculate_all_nucleophilicities(
    structures: List[str],
    gaussian_path: Path,
    verbose: bool = True
) -> List[NucleophilicityResult]:
    """Calculate nucleophilicity for all structures.
    
    Args:
        structures: List of structure names
        gaussian_path: Path to Gaussian output directory
        verbose: If True, print progress
        
    Returns:
        List of NucleophilicityResult objects
    """
    # Get TCE reference HOMO
    tce_path = gaussian_path / "TCE.out"
    tce_orbitals = parse_homo_lumo(tce_path)
    e_homo_tce = tce_orbitals.homo
    
    if verbose:
        print(f"TCE reference HOMO: {e_homo_tce:.3f} eV")
    
    results = []
    for structure in structures:
        try:
            filepath = gaussian_path / f"{structure}.out"
            orbitals = parse_homo_lumo(filepath)
            
            nucleophilicity = calculate_nucleophilicity(orbitals.homo, e_homo_tce)
            
            results.append(NucleophilicityResult(
                name=structure,
                homo=orbitals.homo,
                lumo=orbitals.lumo,
                homo_lumo_gap=orbitals.homo_lumo_gap,
                nucleophilicity=nucleophilicity,
            ))
        except Exception as e:
            print(f"Error processing {structure}: {e}")
    
    if verbose:
        print(f"Calculated nucleophilicity for {len(results)} structures.")
    
    return results

In [ ]:
# Calculate nucleophilicity at two levels of theory
nuc_b3lyp_631g = calculate_all_nucleophilicities(STRUCTURES, GAUSSIAN_B3LYP_631G)
nuc_b3lyp_cbsb7 = calculate_all_nucleophilicities(STRUCTURES, GAUSSIAN_B3LYP_CBSB7)

In [ ]:
# Plot basis set comparison
def plot_basis_set_comparison(
    nuc_631g: List[NucleophilicityResult],
    nuc_cbsb7: List[NucleophilicityResult],
    save_path: Optional[Path] = None
) -> None:
    """Plot nucleophilicity comparison between basis sets.
    
    Args:
        nuc_631g: Results from 6-31G* basis
        nuc_cbsb7: Results from CBSB7 basis
        save_path: If provided, save figure to this path
    """
    # Match results by name
    names_631g = {r.name: r.nucleophilicity for r in nuc_631g}
    
    x_vals, y_vals = [], []
    for r in nuc_cbsb7:
        if r.name in names_631g:
            x_vals.append(names_631g[r.name])
            y_vals.append(r.nucleophilicity)
    
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.scatter(x_vals, y_vals, alpha=0.7, edgecolors="k", linewidths=0.5)
    
    # Add correlation line
    z = np.polyfit(x_vals, y_vals, 1)
    p = np.poly1d(z)
    x_line = np.linspace(min(x_vals), max(x_vals), 100)
    ax.plot(x_line, p(x_line), "--", color="gray", alpha=0.7)
    
    # Calculate R²
    correlation = np.corrcoef(x_vals, y_vals)[0, 1]
    r_squared = correlation ** 2
    
    ax.set_xlabel("Nucleophilicity (B3LYP/6-31G*) [eV]", fontsize=11)
    ax.set_ylabel("Nucleophilicity (B3LYP/CBSB7) [eV]", fontsize=11)
    ax.set_title(f"Basis Set Comparison (R² = {r_squared:.3f})", fontsize=12)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    
    plt.show()


plot_basis_set_comparison(
    nuc_b3lyp_631g, 
    nuc_b3lyp_cbsb7,
    save_path=OUTPUT_DIR / "nucleophilicity_basis_comparison.png"
)

<a id='deprotonation'></a>
## 5. Deprotonation Energy Calculations

Calculate gas-phase deprotonation enthalpies and free energies using G4MP2 composite method:

$$\Delta H_{\text{deprot}} = \left( H_{\text{R}^-} + H_{\text{H}^+} \right) - H_{\text{R-H}}$$

$$\Delta G_{\text{deprot}} = \left( G_{\text{R}^-} + G_{\text{H}^+} \right) - G_{\text{R-H}}$$

In [ ]:
@dataclass
class G4MP2Energies:
    """G4MP2 thermodynamic quantities.
    
    Attributes:
        enthalpy: G4MP2 enthalpy (Hartrees)
        free_energy: G4MP2 free energy (Hartrees)
    """
    enthalpy: float
    free_energy: float

In [ ]:
def parse_g4mp2_energies(filepath: Path) -> G4MP2Energies:
    """Extract G4MP2 enthalpy and free energy from Gaussian output.
    
    Args:
        filepath: Path to G4MP2 Gaussian output file
        
    Returns:
        G4MP2Energies with H and G in Hartrees
        
    Raises:
        ValueError: If G4MP2 energies not found
    """
    with open(filepath, "r") as f:
        for line in f:
            if line.startswith(" G4MP2 Enthalpy="):
                # Format: " G4MP2 Enthalpy=    -XXX.XXXXXX  G4MP2 Free Energy=    -XXX.XXXXXX"
                enthalpy = float(line[18:37].strip())
                free_energy = float(line[63:].strip())
                return G4MP2Energies(enthalpy=enthalpy, free_energy=free_energy)
    
    raise ValueError(f"G4MP2 energies not found in {filepath}")

In [ ]:
@dataclass
class DeprotonationResult:
    """Deprotonation thermodynamics for a molecule.
    
    Attributes:
        name: Molecule identifier
        delta_h: Deprotonation enthalpy (kcal/mol)
        delta_g: Deprotonation free energy (kcal/mol)
    """
    name: str
    delta_h: float
    delta_g: float

In [ ]:
def calculate_deprotonation_energies(
    structures: List[str],
    g4mp2_path: Path,
    verbose: bool = True
) -> List[DeprotonationResult]:
    """Calculate deprotonation energies for all structures.
    
    Uses capped structures (blocking group + H) for the calculation.
    
    Args:
        structures: List of base structure names (without _cap suffix)
        g4mp2_path: Path to G4MP2 output directory
        verbose: If True, print progress
        
    Returns:
        List of DeprotonationResult objects
    """
    # Get H+ reference energy
    h_plus = parse_g4mp2_energies(g4mp2_path / "H+.out")
    
    # Filter to capping agent structures only
    base_structures = [s for s in structures if not s.endswith("_cap")]
    
    results = []
    for structure in base_structures:
        try:
            parent = parse_g4mp2_energies(g4mp2_path / f"{structure}_cap.out")
            anion = parse_g4mp2_energies(g4mp2_path / f"{structure}_cap_anion.out")
            
            # ΔH = (H_anion + H_H+) - H_parent
            delta_h = ((anion.enthalpy + h_plus.enthalpy) - parent.enthalpy) * HARTREE_TO_KCAL
            delta_g = ((anion.free_energy + h_plus.free_energy) - parent.free_energy) * HARTREE_TO_KCAL
            
            results.append(DeprotonationResult(
                name=structure,
                delta_h=delta_h,
                delta_g=delta_g,
            ))
        except FileNotFoundError as e:
            if verbose:
                print(f"Skipping {structure}: {e}")
        except Exception as e:
            print(f"Error processing {structure}: {e}")
    
    if verbose:
        print(f"Calculated deprotonation energies for {len(results)} structures.")
    
    return results

In [ ]:
# Calculate deprotonation energies
deprot_results = calculate_deprotonation_energies(STRUCTURES, GAUSSIAN_G4MP2)

<a id='export'></a>
## 6. Export Results

Export all calculated descriptors to CSV files for further analysis.

In [ ]:
def export_bebop_results(
    bebop_results: List[BEBOPResult],
    nuc_results: List[NucleophilicityResult],
    output_path: Path
) -> None:
    """Export combined BEBOP and nucleophilicity results to CSV.
    
    Args:
        bebop_results: BEBOP calculation results
        nuc_results: Nucleophilicity results
        output_path: Output CSV file path
    """
    # Create lookup for nucleophilicity
    nuc_lookup = {r.name: r for r in nuc_results}
    
    header = (
        "structure,bond1,bond2,bond_gross1,bond_net1,bond_gross2,bond_net2,"
        "C_hyb_bond1,NorO_hyb_bond1,N_hyb_bond2,resonance,"
        "homo_b3lyp,lumo_b3lyp,homo_lumo_gap,nucleophilicity\n"
    )
    
    with open(output_path, "w") as f:
        f.write(header)
        
        for bebop in bebop_results:
            nuc = nuc_lookup.get(bebop.name)
            if nuc is None:
                continue
            
            row = (
                f"{bebop.name},{bebop.bond_co},{bebop.bond_nh},"
                f"{bebop.gross_be_co},{bebop.net_be_co},"
                f"{bebop.gross_be_nh},{bebop.net_be_nh},"
                f"{bebop.hybridization[0]},{bebop.hybridization[1]},{bebop.hybridization[2]},"
                f"{bebop.resonance_energy},"
                f"{nuc.homo},{nuc.lumo},{nuc.homo_lumo_gap},{nuc.nucleophilicity:.2f}\n"
            )
            f.write(row)
    
    print(f"Exported BEBOP results to {output_path}")

In [ ]:
def export_nucleophilicity(
    nuc_results: List[NucleophilicityResult],
    output_path: Path
) -> None:
    """Export nucleophilicity results to CSV.
    
    Args:
        nuc_results: Nucleophilicity results
        output_path: Output CSV file path
    """
    with open(output_path, "w") as f:
        f.write("structure,nucleophilicity\n")
        for r in nuc_results:
            f.write(f"{r.name},{r.nucleophilicity:.2f}\n")
    
    print(f"Exported nucleophilicity to {output_path}")

In [ ]:
def export_deprotonation(
    deprot_results: List[DeprotonationResult],
    output_path: Path
) -> None:
    """Export deprotonation energies to CSV.
    
    Args:
        deprot_results: Deprotonation results
        output_path: Output CSV file path
    """
    with open(output_path, "w") as f:
        f.write("structure,delta_h_kcal_mol,delta_g_kcal_mol\n")
        for r in deprot_results:
            f.write(f"{r.name},{r.delta_h:.2f},{r.delta_g:.2f}\n")
    
    print(f"Exported deprotonation energies to {output_path}")

In [ ]:
# Export all results
export_bebop_results(
    bebop_results, 
    nuc_b3lyp_cbsb7, 
    OUTPUT_DIR / "bebop_descriptors.csv"
)

export_nucleophilicity(
    nuc_b3lyp_631g, 
    OUTPUT_DIR / "nucleophilicity_b3lyp_631g.csv"
)

export_deprotonation(
    deprot_results, 
    OUTPUT_DIR / "deprotonation_energies_g4mp2.csv"
)

---

## Summary

This notebook calculated the following electronic descriptors for deblocking reactions:

1. **Bond energies** (BEBOP): Gross and net bond energies for key bonds
2. **Hybridization energies** (BEBOP): sp-character of atoms in reactive bonds
3. **Resonance energies** (BEBOP): Stabilization from $\pi$-delocalization
4. **Nucleophilicity** (QM/B3LYP): Domingo's group $N$ index relative to TCE
5. **HOMO-LUMO Gaps** (QM/B3LYP): HOMO-LUMO Gap Energy 
6. **Deprotonation energies** (QM/G4MP2): Gas-phase $\Delta H$ and $\Delta G$

Results are exported to CSV files in the `./results/` directory.

---

## License

This code is released under the MIT License. See LICENSE file for details.